In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

# Load the dataset
data = pd.read_csv("dataset.csv")

# Extract features and labels
image_paths = data["image_path"]
labels = data["artist_label"]

num_artists = len(os.listdir('dataset/'))

# Function to load and preprocess images while preserving aspect ratio
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path)  # Load image without resizing
        img = image.img_to_array(img)
        
        # Calculate target width and height while preserving aspect ratio
        height, width, _ = img.shape
        target_height, target_width = target_size
        
        # Resize image with aspect ratio preserved
        if width > height:
            new_width = target_width
            new_height = int((height / width) * target_width)
        else:
            new_height = target_height
            new_width = int((width / height) * target_height)
        
        img_resized = image.array_to_img(img)
        img_resized = img_resized.resize((new_width, new_height))

        # Pad the image to target size
        img_padded = np.zeros((target_height, target_width, 3))  # Create a black canvas
        img_padded[:new_height, :new_width, :] = np.array(img_resized)  # Place the resized image on the canvas
        
        images.append(img_padded)
    
    return np.array(images)

# Load and preprocess images
X = load_and_preprocess_images(image_paths)
X = X / 255.0  # Normalize the images to [0, 1] range

# Convert labels to numpy array and one-hot encode them
y = np.array(labels)
y = to_categorical(y, num_classes=num_artists) 

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a Convolutional Neural Network (CNN) model for multi-class classification
model = tf.keras.Sequential([
    # First convolutional block
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Second convolutional block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Third convolutional block
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Flatten the output
    tf.keras.layers.Flatten(),
    
    # Fully connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Output layer with num_artists units for multi-class classification
    tf.keras.layers.Dense(num_artists, activation='softmax')  # num_artists classes, softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Categorical cross-entropy for multi-class classification
              metrics=['accuracy'])

# Data augmentation to improve generalization
datagen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the data augmentation generator
datagen.fit(X_train)

# Train the model with augmented data
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

# Predict on the test set
y_pred = np.argmax(model.predict(X_test), axis=1)  # Convert softmax probabilities to class labels

# Get the actual class labels (as integers)
y_true = np.argmax(y_test, axis=1)


c:\Users\zq\Documents\GitHub\yukien\venv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
c:\Users\zq\Documents\GitHub\yukien\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


c:\Users\zq\Documents\GitHub\yukien\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


67/67 ━━━━━━━━━━━━━━━━━━━━ 14s 198ms/step - accuracy: 0.4890 - loss: 1.1018 - val_accuracy: 0.6906 - val_loss: 0.8055
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 13s 197ms/step - accuracy: 0.6598 - loss: 0.8766 - val_accuracy: 0.6528 - val_loss: 0.8301
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 13s 194ms/step - accuracy: 0.6843 - loss: 0.7789 - val_accuracy: 0.7321 - val_loss: 0.6658
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 14s 202ms/step - accuracy: 0.7165 - loss: 0.7303 - val_accuracy: 0.7509 - val_loss: 0.7225
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 14s 202ms/step - accuracy: 0.7285 - loss: 0.7129 - val_accuracy: 0.7679 - val_loss: 0.7724
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 14s 205ms/step - accuracy: 0.7398 - loss: 0.7060 - val_accuracy: 0.7509 - val_loss: 0.6997
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 14s 205ms/step - accuracy: 0.7402 - loss: 0.6682 - val_accuracy: 0.7604 - val_loss: 0.6386
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 14s 204ms/step - accuracy: 0.7539 - loss: 0.6461 - val_accuracy: 0.792

In [ ]:

# Classification report for performance evaluation
report = classification_report(y_true, y_pred, 
                              target_names=[f'Class {i}' for i in np.unique(y_true)])
print(report)


              precision    recall  f1-score   support

     Class 0       0.80      0.90      0.85       180
     Class 1       0.64      0.67      0.66        43
     Class 2       0.83      0.85      0.84       264
     Class 3       0.75      0.21      0.33        43

    accuracy                           0.80       530
   macro avg       0.76      0.66      0.67       530
weighted avg       0.80      0.80      0.79       530